In [34]:
import numpy as np
import pandas as pd
import requests
import time
from urllib3 import Timeout, PoolManager
from pathlib import Path
import os
import sys
import math

In [4]:
sys.path.append(os.path.abspath('../Helper Functions'))
from objectives import *

In [5]:
timeout = Timeout(connect=10.0, read=15.0)      # Set up timeout to avoid stucking in the request and forcing to restart the kernel
http = PoolManager(timeout=timeout)

In [6]:
dev_key = '' # Enter your roit dev key

In [37]:
games_df = pd.DataFrame()   # Initialize an empty DataFrame to add our data

In [8]:
matches_id = []     # Used to keep track of the matches_id to avoid redundant ids, intially we have no matches_id

In [8]:
# In case we have collected some matches_id, we can load them to check about redundancy
matches_id = []

file_path = "" # Name of the file containg the matches_id

# Append these ids to a list
with open(file_path, "r") as file:
    for line in file:
        matches_id.append(line.strip())
        


FileNotFoundError: [Errno 2] No such file or directory: ''

In [38]:
# Get the csv file created in Puuid folder, typically named 'region' + 'RANK' ex: 'EUW MASTERS.csv' 
parent_dir = os.path.abspath(os.path.join(os.getcwd(), '..'))
main_file_path = os.path.join(parent_dir, 'Puuid\\.csv')    # Add the name of the csv file here
games = pd.read_csv(main_file_path,index_col=False)

In [39]:
games

,MATCH ID
0,EUW1_7234969643
1,EUW1_7234956683
2,EUW1_7234935840
3,EUW1_7234902774
4,EUW1_7234038609
...,...
21583,NA1_5154071350
21584,NA1_5154061093
21585,NA1_5154041727
21586,NA1_5153883983


In [40]:
puuids = games['Puuid'].to_list()  # Extract the puuids in a list to iterate over

In [41]:
region = 'europe' # Enter the region

In [10]:
for puuid in puuids:
    try:
        print(puuid)
        # Consider 47 games starting from game zero
        # response = http.request('GET','https://europe.api.riotgames.com/lol/match/v5/matches/by-puuid/'+ puuid +'/ids?start=0&count=47&api_key='+dev_key)   # 'europe' is changed according to the region 
        timeline_matches,respons = get_match_ids(puuid, region, dev_key)
    
        for match in timeline_matches:
                timeline_match = http.request('GET',f'https://{region}.api.riotgames.com/lol/match/v5/matches/'+ match +'/timeline?api_key='+dev_key).json()   # 'europe' is changed according to the region 
                match_data = http.request('GET',f'https://{region}.api.riotgames.com/lol/match/v5/matches/' + match +'?api_key='+dev_key).json()   # 'europe' is changed according to the region 
                players_counuter = sum(1 for player in match_data['info']['participants'] if 'challenges' in player)    # Counting number of players to check for the 10 players
                games_dic = {}
                # Check for Unique match_id, 10 players exist, gameModee is CLASSIC (5v5 in Summoner's rift)
                if match not in matches_id and players_counuter == 10 and match_data['info']['gameMode'] == 'CLASSIC':
                    # first_tower = False
                    games_dic, players_id, flag = championName_playerId(match_data)  # Define games_dic and players_id as two dictionaries for that game
                    if flag == False:   # Skipping matches with missinng role(s)
                        continue
                    # Intilaization
                    red_towers, blue_towers ,red_top_inhib , red_mid_inhib ,red_bot_inhib, blue_top_inhib, blue_mid_inhib ,blue_bot_inhib,red_fire_dragons,red_water_dragons,red_earth_dragons,red_air_dragons,red_hextech_dragons,red_chemtech_dragons,red_elder_dragons,blue_fire_dragons,blue_water_dragons,blue_earth_dragons,blue_air_dragons,blue_hextech_dragons,blue_chemtech_dragons,blue_elder_dragons,red_barons,blue_barons,red_rift,blue_rift,red_hordes,blue_hordes, kills, deaths, assists, kda, red_first_tower, red_first_blood, blue_first_tower, blue_first_blood = intialization(players_id)   # Intializations
                    for i in range(len(timeline_match['info']['frames'])):
                        if i == 0:      # Start game frame is skipped
                            continue
                        for j in range(len(timeline_match['info']['frames'][i]['events'])):
                            # try:
                                event_type = timeline_match['info']['frames'][i]['events'][j]['type']
                                participant_frame = timeline_match['info']['frames'][i]['participantFrames']
                                # Buildings, which are turrets and inhibs
                                if  event_type == 'BUILDING_KILL':
                                    # Turrets
                                    if timeline_match['info']['frames'][i]['events'][j]['buildingType'] == "TOWER_BUILDING":
                                        lane_type = timeline_match['info']['frames'][i]['events'][j]['laneType']
                                        tower_type = timeline_match['info']['frames'][i]['events'][j]['towerType']
                                        tower = lane_type + " " +tower_type    
                                        # if first_tower == False:
                                        #     first_tower = True
                                        # else:
                                        #     first_tower = False
    
                                        if timeline_match['info']['frames'][i]['events'][j]['teamId'] == 200: # red team tower destroyed
                                            try:
                                                red_towers.remove('RED ' + tower)
                                            except:
                                                print('RED ' + tower)
                                                print(red_towers)
                                            # if first_tower == True:
                                            #     blue_first_tower = True
    
                                        
                                        elif timeline_match['info']['frames'][i]['events'][j]['teamId'] == 100: # blue team tower destroyed
                                            try:
                                                blue_towers.remove('BLUE ' + tower)     # Remove destroyed tower from the list
                                            except:
                                                print('BLUE ' + tower)
                                                print(blue_towers)

                                            # if first_tower == True:
                                            #     red_first_tower = True
    
                                        
                                    # Inhibs
                                    elif timeline_match['info']['frames'][i]['events'][j]['buildingType'] == 'INHIBITOR_BUILDING':
                                        lane_type = timeline_match['info']['frames'][i]['events'][j]['laneType']
                                        if timeline_match['info']['frames'][i]['events'][j]['teamId'] == 200: # red team inhib destroyed, so we can increase blue team inhib counts
                                            if lane_type == "TOP_LANE":
                                                blue_top_inhib = 0
                                            elif lane_type == "MID_LANE":
                                                blue_mid_inhib = 0
                                            elif lane_type == "BOT_LANE":
                                                blue_bot_inhib = 0
                                            else:
                                                print("ERROR Inhib")
                                        
                                        elif timeline_match['info']['frames'][i]['events'][j]['teamId'] == 100: # blue team inhib destroyed, so we can increase red team inhib counts
                                            if lane_type == "TOP_LANE":
                                                red_top_inhib = 0
                                            elif lane_type == "MID_LANE":
                                                red_mid_inhib = 0
                                            elif lane_type == "BOT_LANE":
                                                red_bot_inhib = 0
                                            else:
                                                print("ERROR Inhib")
    
                                        
    
                                # Elite Monsters, which include dragon, grubs, rift, baron
                                elif event_type == 'ELITE_MONSTER_KILL':
                                    # Dragons
                                    if timeline_match['info']['frames'][i]['events'][j]['monsterType'] == 'DRAGON':
                                        dragon_type = timeline_match['info']['frames'][i]['events'][j]['monsterSubType']
                                        if timeline_match['info']['frames'][i]['events'][j]['killerTeamId'] == 200: # red team killed the dragon
                                            red_air_dragons, red_fire_dragons, red_earth_dragons, red_water_dragons, red_hextech_dragons, red_chemtech_dragons, red_elder_dragons = check_dragon_type(red_air_dragons, red_fire_dragons, red_earth_dragons, red_water_dragons, red_hextech_dragons, red_chemtech_dragons, red_elder_dragons,dragon_type)
                                        
                                        elif timeline_match['info']['frames'][i]['events'][j]['killerTeamId'] == 100: # blue team killed the dragon
                                            blue_air_dragons, blue_fire_dragons, blue_earth_dragons, blue_water_dragons, blue_hextech_dragons, blue_chemtech_dragons, blue_elder_dragons = check_dragon_type(blue_air_dragons, blue_fire_dragons, blue_earth_dragons, blue_water_dragons, blue_hextech_dragons, blue_chemtech_dragons,blue_elder_dragons,dragon_type)
    
                                        
    
                                    
                                    # Barons
                                    elif timeline_match['info']['frames'][i]['events'][j]['monsterType'] == 'BARON_NASHOR':
                                        if timeline_match['info']['frames'][i]['events'][j]['killerTeamId'] == 200: # red team killed the baron
                                            red_barons += 1
                                        elif timeline_match['info']['frames'][i]['events'][j]['killerTeamId'] == 100: # blue team killed the baron
                                            blue_barons += 1
    
                                        
    
                                    # RiftHerald
                                    elif timeline_match['info']['frames'][i]['events'][j]['monsterType'] == 'RIFTHERALD':
                                        if timeline_match['info']['frames'][i]['events'][j]['killerTeamId'] == 200: # red team killed the rift
                                            red_rift = True
                                        elif timeline_match['info']['frames'][i]['events'][j]['killerTeamId'] == 100: # blue team killed the rift
                                            blue_rift = True
                                        
                                        
    
                                        
    
                                    # VoidGrubs
                                    elif timeline_match['info']['frames'][i]['events'][j]['monsterType'] == 'HORDE':
                                        if timeline_match['info']['frames'][i]['events'][j]['killerTeamId'] == 200: # red team killed one horde
                                            red_hordes += 1
                                        elif timeline_match['info']['frames'][i]['events'][j]['killerTeamId'] == 100: # blue team killed one horde
                                            blue_hordes += 1
                                
    
    
                                # Kills, Deaths, and Assists
                                elif event_type == 'CHAMPION_KILL':
                                    killer_id = timeline_match['info']['frames'][i]['events'][j]['killerId']
                                    victim_id = timeline_match['info']['frames'][i]['events'][j]['victimId']
                                    try:
                                        assistors_list = timeline_match['info']['frames'][i]['events'][j]['assistingParticipantIds']
                                    except:
                                        assistors_list = []
                                        # print("No Assistors in that kill")
    
                                    kills, daths, assists = kills_deaths_assists(killer_id, victim_id, assistors_list,kills,deaths,assists)
                                    kda = compute_kda(kills,deaths,assists, kda)      
    
                                # First blood
                                # elif event_type == "CHAMPION_SPECIAL_KILL" and timeline_match['info']['frames'][i]['events'][j]['killType'] == 'KILL_FIRST_BLOOD':
                                #     killer_id = timeline_match['info']['frames'][i]['events'][j]['killerId']
                                #     if killer_id == 0:
                                #         pass
                                #     elif killer_id > 0 and killer_id <= 5:  # Blue team killer
                                #         blue_first_blood = True
                                #     elif killer_id > 0 and killer_id >= 6:
                                #         red_first_blood = True
    
    
                        # Lane minions and Jungle minions
                        players_cs_dic = players_cs(timeline_match['info']['frames'][i]['participantFrames'], players_id)
    
                        # Champions Level
                        levels = champions_level(timeline_match['info']['frames'][i]['participantFrames'], players_id)
    
                        # Items
                        items = collect_items(match_data, players_id)
    
                    
    
                        # UPDATING GAME_DIC GAME DATA
    
                        # Add champions level
                        for key in list(levels.keys()):
                            games_dic.update({
                                key: levels[key]
                        })
                            
                        # Add champions total CS
                        for key in list(players_cs_dic.keys()):
                            games_dic.update({
                                key: players_cs_dic[key]
                                # round(players_cs_dic[key]/i,2)     # get CS/min rather than CS
                            })
    
                        # Add Items
                        for key in list(items.keys()):
                            games_dic.update({
                                key : items[key]
                            })
                        
    
                        # Add red and blue turrets
                        games_dic.update({"RED Turrets": red_towers,
                                        "BLUE Turrets": blue_towers})
    
                        # Add red and blue inhibs
                        games_dic.update({"RED TOP INHIB":red_top_inhib,
                                            "RED MID INHIB": red_mid_inhib,
                                            "RED_BOT_INHIB": red_bot_inhib,
                                            "BLUE TOP INHIB": blue_top_inhib,
                                            "BLUE MID INHIB": blue_mid_inhib,
                                            "BLUE_BOT_INHIB":  blue_bot_inhib})
    
                        # Add red team and blue team killed dragons
                        games_dic.update({
                            "RED FIRE DRAGON":red_fire_dragons,
                            "RED AIR DRAGON": red_air_dragons,
                            "RED EARTH DRAGON": red_earth_dragons,
                            "RED WATER DRAGON": red_water_dragons,
                            "RED HEXTECH DRAGON": red_hextech_dragons,
                            "RED CHEMTECH DRAGON": red_chemtech_dragons,
                            "RED ELDER DRAGON": red_elder_dragons
                        })
    
                        games_dic.update({
                            "BLUE FIRE DRAGON":blue_fire_dragons,
                            "BLUE AIR DRAGON": blue_air_dragons,
                            "BLUE EARTH DRAGON": blue_earth_dragons,
                            "BLUE WATER DRAGON": blue_water_dragons,
                            "BLUE HEXTECH DRAGON": blue_hextech_dragons,
                            "BLUE CHEMTECH DRAGON": blue_chemtech_dragons,
                            "BLUE ELDER DRAGON": blue_elder_dragons
                        })
    
                        # if i > 7:   # No Baron or Rift before min 7, so this will always be False
                        # Add red team and blue team killed barons
                        games_dic.update({
                            "RED BARON": red_barons,
                            "BLUE BARON": blue_barons
                        })
    
                        # Add red team and blue team defeated rift
                        games_dic.update({
                            "RED RIFT": red_rift,
                            "BLUE RIFT": blue_rift
                        })
    
                        # Add red team and blue team killed VoidGrubs
                        games_dic.update({
                            "RED GRUBS": red_hordes,
                            "BLUE GRUBS": blue_hordes
                        })
    
    
                        # Add each player KDA
                        for k in range(10):
                            if k <= 4:    # BLUE TEAM
                                games_dic.update({f"BLUE {match_data['info']['participants'][k]['teamPosition']} KDA": kda[k+1]})
    
                            elif k >= 5:
                                games_dic.update({f"RED {match_data['info']['participants'][k]['teamPosition']} KDA": kda[k+1]})
    
    
                        games_dic.update({
                            "TIME STAMP" : 0
                        })
    
                        if match_data['info']['teams'][0]['teamId'] == 100: # Blue team
                            games_dic.update({
                                "BLUE WIN" : match_data['info']['teams'][0]['win']
                            })
            


                    games_dic['TIME STAMP'] = timeline_match['info']['frames'][-1]['events'][-1]['timestamp']
                    games_dic['MATCH ID'] = timeline_match['metadata']['matchId']   
                    games_dic['RANK'] =  games['tier'][0] + ' ' + games['rank'][0]   
                    games_df = pd.concat([games_df, pd.DataFrame([games_dic])],ignore_index=True)
    
                    matches_id.append(match)
    
        print(len(games_df))
        print(puuids.index(puuid))
        games = games[games['Puuid'] != puuid]

    except Exception as e:
        if response.status == 403:       # In case you are using riot key that expires, we break when the key is expired
            break
        print("ERROR", puuid, f'exception {e}')       # Check for puuid causing error and observe the response status code
        print(response.status)
        continue
    delay_with_interrupt(122)       # every almost 100 requests we sleep for 2 minutes to avoid ratelimit excess

Ycb0a14vaclWhdluM6dvof-UOqGlFZ61uQVrM925XjGtABfHlK5e5zy3k_0iFy7R6gHB5YSpisEcKg
Fetched 47 matches for queue ID 420.
Total matches fetched: 47
BLUE MID_LANE NEXUS_TURRET
[]
47
0


KeyboardInterrupt: 

In [13]:
games.to_csv(main_file_path)   # Save the updated version of games dataframe, change the EUW MASTER.csv to you file name

In [14]:
games_df.to_csv('games_df_EUW.csv', index=False)    # Save this file to concate with other files 